In [1]:
from pyspark.sql import SparkSession
sparks = SparkSession \
        .builder \
        .appName("Kafka Streaming Sink") \
        .master("local[3]") \
        .getOrCreate()

In [2]:
spark

In [3]:
kafkaDFS = sparks.readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "iabd-virtualbox:9092") \
        .option("subscribe", "facturas") \
        .option("startingOffsets", "earliest") \
        .load()

In [4]:
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType, IntegerType, ArrayType
esquema = StructType([
    StructField("InvoiceNumber", StringType()),
    StructField("CreatedTime", LongType()),
    StructField("StoreID", StringType()),
    StructField("PosID", StringType()),
    StructField("CashierID", StringType()),
    StructField("CustomerType", StringType()),
    StructField("CustomerCardNo", StringType()),
    StructField("TotalAmount", DoubleType()),
    StructField("NumberOfItems", IntegerType()),
    StructField("PaymentMethod", StringType()),
    StructField("CGST", DoubleType()),
    StructField("SGST", DoubleType()),
    StructField("CESS", DoubleType()),
    StructField("DeliveryType", StringType()),
    StructField("DeliveryAddress", StructType([
        StructField("AddressLine", StringType()),
        StructField("City", StringType()),
        StructField("State", StringType()),
        StructField("PinCode", StringType()),
        StructField("ContactNumber", StringType())
    ])),
    StructField("InvoiceLineItems", ArrayType(StructType([
        StructField("ItemCode", StringType()),
        StructField("ItemDescription", StringType()),
        StructField("ItemPrice", DoubleType()),
        StructField("ItemQty", IntegerType()),
        StructField("TotalValue", DoubleType())
    ]))),
])

### Gestionamos las facturas - caso 03

In [6]:
from pyspark.sql.functions import from_json, col
valueDF = kafkaDFS.select(from_json(col("value").cast("string"), esquema).alias("value"))
valueDF.printSchema()

root
 |-- value: struct (nullable = true)
 |    |-- InvoiceNumber: string (nullable = true)
 |    |-- CreatedTime: long (nullable = true)
 |    |-- StoreID: string (nullable = true)
 |    |-- PosID: string (nullable = true)
 |    |-- CashierID: string (nullable = true)
 |    |-- CustomerType: string (nullable = true)
 |    |-- CustomerCardNo: string (nullable = true)
 |    |-- TotalAmount: double (nullable = true)
 |    |-- NumberOfItems: integer (nullable = true)
 |    |-- PaymentMethod: string (nullable = true)
 |    |-- CGST: double (nullable = true)
 |    |-- SGST: double (nullable = true)
 |    |-- CESS: double (nullable = true)
 |    |-- DeliveryType: string (nullable = true)
 |    |-- DeliveryAddress: struct (nullable = true)
 |    |    |-- AddressLine: string (nullable = true)
 |    |    |-- City: string (nullable = true)
 |    |    |-- State: string (nullable = true)
 |    |    |-- PinCode: string (nullable = true)
 |    |    |-- ContactNumber: string (nullable = true)
 |    |

In [7]:
from pyspark.sql.functions import expr

explodeDF = valueDF.selectExpr("value.InvoiceNumber", "value.CreatedTime",
    "value.StoreID", "value.PosID", "value.CustomerType",
    "value.PaymentMethod", "value.DeliveryType", "value.DeliveryAddress.City",
    "value.DeliveryAddress.State", "value.DeliveryAddress.PinCode",
    "explode(value.InvoiceLineItems) as LineItem")

limpioDF = explodeDF \
    .withColumn("ItemCode", expr("LineItem.ItemCode")) \
    .withColumn("ItemDescription", expr("LineItem.ItemDescription")) \
    .withColumn("ItemPrice", expr("LineItem.ItemPrice")) \
    .withColumn("ItemQty", expr("LineItem.ItemQty")) \
    .withColumn("TotalValue", expr("LineItem.TotalValue")) \
    .drop("LineItem")

In [8]:
facturaWriterQuery  = limpioDF.writeStream \
    .format("json") \
    .queryName("Facturas Kafka Writer") \
    .outputMode("append") \
    .option("path", "salida") \
    .option("checkpointLocation", "chk-point-dir-03") \
    .trigger(processingTime="1 minute") \
    .start()

### Gestionamos las notificaciones - caso 04

In [9]:
from pyspark.sql.functions import from_json, col, expr
valueDF = kafkaDFS.select(from_json(col("value").cast("string"), esquema).alias("value"))

notificationDF = valueDF.select("value.InvoiceNumber", "value.CustomerCardNo", "value.TotalAmount") \
    .withColumn("LoyaltyPoints", expr("TotalAmount * 0.2"))

# Transformamos las cuatro columnas en lo que espera Kafka, un par de (key, value)
kafkaTargetDF = notificationDF.selectExpr("InvoiceNumber as key",
        """to_json(named_struct(
        'CustomerCardNo', CustomerCardNo,
        'TotalAmount', TotalAmount,
        'EarnedLoyaltyPoints', TotalAmount * 0.2)) as value""")

In [10]:
notificacionWriterQuery = kafkaTargetDF \
    .writeStream \
    .queryName("Notificaciones Writer") \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "iabd-virtualbox:9092") \
    .option("topic", "notificaciones") \
    .outputMode("append") \
    .option("checkpointLocation", "chk-point-dir-034") \
    .start()

### Lanzamos todas las querys y quedamos a la espera

In [ ]:
sparks.streams.awaitAnyTermination()